In [1]:
import torch as tc
import pandas as pd
import numpy as np
from src.bbox import BBox
import open3d as otd
from tqdm import tqdm
import matplotlib.pyplot as plt
otd_vector3d = otd.utility.Vector3dVector
from src.star.star import STAR
from src.curve_utils import CurveUtils
from src.curve_generator import CurveGenerator
from src.mesh_manipulation import save_obj
device = tc.device("cuda" if tc.cuda.is_available() else "cpu")
genders = ['female', 'male']
male = ['male']
female = ['female']

In [2]:
their_semantics = [
    'Bust girth',
    'Waist girth',
    'Hip girth',
    'Thigh girth R',
    'Upper arm girth R',
    'Neck girth',
    'Back neck point to waist',
    'Height (m)',
]
our_semantic = [
    'bust_chest_girth', # 5.3.4
    'waist_girth', # 5.3.10
    'hip_girth', # 5.3.13
    'thigh_girth', # 5.3.20
    'upper_arm_girth', # 5.3.16
    'neck_girth', # 5.3.2
    'neck_waist',
    'stature', # 5.1.1
]

curve_index = {
    'neck_girth':4, # 5.3.2
    'bust_chest_girth': 0, # 5.3.4
    'waist_girth': 1, # 5.3.10
    'hip_girth': 1, # 5.3.13
    'upper_arm_girth': 3, # 5.3.16
    'thigh_girth': 2, # 5.3.20
    #'neck_waist': 5
}

In [3]:
subdivided_bodies = tc.load('data/subdivided_bodies.pt')
measures = pd.read_pickle(f'data/cleaned_measures.zip')
measures.index = measures['Subject']
measures = measures[measures['Measuring station'] == "MOVE4D"]

mfd_gender_measures = dict()
for gender in genders:
    mfd_gender_measures[gender] = measures[measures['Sex'] == gender]
    mfd_gender_measures[gender] = mfd_gender_measures[gender][their_semantics]
    mfd_gender_measures[gender].columns = our_semantic
    mfd_gender_measures[gender]['stature'] *= 1000

In [4]:
selected_subjects = dict()
selected_measures = dict()

for gender in genders:
    gender_measures = measures[measures['Sex'] == gender]
    temp_measures = gender_measures[their_semantics].iloc[::2]
    selected_subjects[gender] = 'IEEEP2_07' if gender == 'female' else 'IEEEP2_04'
    selected_measures[gender] = temp_measures.loc[selected_subjects[gender]]
    selected_subjects[gender] = temp_measures.index.get_loc(selected_subjects[gender])*2
    selected_measures[gender].index = our_semantic
    selected_measures[gender]['stature'] *= 1000

In [5]:
body = subdivided_bodies['vertices']['female'][selected_subjects['female']]
template = subdivided_bodies['faces']['female']+1
# save_obj(path="output/female_body.obj", pontos=body, faces=template)

In [6]:
gender_curves = dict()
for gender in genders:
    print(f'SEGMENTING {gender.upper()} BODIES', end='')
    faces = subdivided_bodies['faces'][gender].to(device)
    bodies = subdivided_bodies['vertices'][gender]
    body = bodies[selected_subjects[gender]].to(device)
    measures = selected_measures[gender]
    result = CurveGenerator.get_curves(body, measures, faces, device, gender)
    gender_curves[gender] = result[0]
tc.save(gender_curves, "data/gender_curves.zip")

SEGMENTING FEMALE BODIES

processing body:  98%|█████████▊| 44/45 [00:44<00:01,  1.02s/it]


SEGMENTING MALE BODIES

processing body:  98%|█████████▊| 44/45 [00:48<00:01,  1.10s/it]


In [7]:
gender_measures = dict()
for gender in genders:
    print(f'MEASURING {gender.upper()} BODIES', end='')
    gender_measures[gender] = [[]]*5
    for segments_index, curves_segments in enumerate(gender_curves[gender]):
        gender_measures[gender][segments_index] = []
        for curves_index, curves in enumerate(tqdm(curves_segments)): # bust, torso, leg, arm, neck
            
            gender_measures[gender][segments_index].append([])
            for body in subdivided_bodies['vertices'][gender]:
                position = CurveUtils.generate_positions(tc.FloatTensor(curves).to(device), body.to(device))
                calculated_measures = CurveUtils.calculate_distances(position)
                gender_measures[gender][segments_index][curves_index].append(calculated_measures)

            calculated_measures = gender_measures[gender][segments_index][curves_index]
            gender_measures[gender][segments_index][curves_index] = tc.FloatTensor(calculated_measures)
        gender_measures[gender][segments_index] = tc.row_stack(gender_measures[gender][segments_index])

tc.save(gender_measures, "data/calculated_measures.zip")


MEASURING FEMALE BODIES

100%|██████████| 640/640 [00:52<00:00, 12.30it/s]


MEASURING MALE BODIES

100%|██████████| 680/680 [00:49<00:00, 13.79it/s]


In [8]:
gender_calculated_measures = tc.load("data/calculated_measures.zip")

In [9]:
gender_measures_wc = dict()
for gender in genders:
    gender_measures_wc[gender] = []
    for index in range(0,5):
        gender_measures_wc[gender].append(gender_measures[gender][index])
        if index == 1:
            gender_measures_wc[gender].append(gender_measures[gender][index])
gender_measures = gender_measures_wc

In [10]:
best_gender_measures = dict() ## caso médio
for idx, gender in enumerate(genders):
    best_gender_measures[gender] = []
    for index, curve in enumerate(our_semantic[:-2]):
        measured = gender_measures[gender][index].T[::2].T
        ground_truth = mfd_gender_measures[gender][curve][::2]
        result = (measured - tc.FloatTensor(ground_truth/10).unsqueeze(0)).abs()
        min_rows_values = result.mean(1)
        min_columns_values, min_columns_indices = min_rows_values.min(0)
        best = min_columns_indices
        best_gender_measures[gender].append((
            best.numpy(),
            result[best].min().numpy(),
            result[best].max().numpy(),
            result[best].mean().numpy(),
            result[best].std().numpy()
        ))
gender_results = {
    'male': pd.DataFrame(best_gender_measures['male'], columns=['best','min', 'max', 'mean', 'std'], index=our_semantic[:-2]),
    'female': pd.DataFrame(best_gender_measures['female'], columns=['best','min', 'max', 'mean', 'std'], index=our_semantic[:-2])
}
print("male errors:")
print(gender_results['male'])
print("\nfemale errors:")
print(gender_results['female'])

tc.save(gender_results, "data/best_gender_curves.zip")

male errors:
                 best           min        max        mean        std
bust_chest_girth   19   0.048316956   4.917015   1.3688946  1.0689367
waist_girth        33   0.008003235  2.1719513   0.6310151  0.5437107
hip_girth          13  0.0023117065  1.2593384  0.45648277  0.3605611
thigh_girth        54   0.004257202  1.8026123  0.70295495  0.5112756
upper_arm_girth    38   0.002937317  0.7723694    0.318335  0.2103261
neck_girth        496  0.0025863647  2.4173927   0.7392074  0.5782629

female errors:
                 best          min        max        mean         std
bust_chest_girth   14   0.16119385   4.739174   1.8618696   1.1967944
waist_girth        30  0.014572144  3.6633835   1.0967818   0.9472004
hip_girth          10  0.057029724  5.4276123   1.2659556   1.1800628
thigh_girth        47  0.019634247  3.2767715  0.86906433  0.68342984
upper_arm_girth    34  0.030994415  1.6408348   0.6151817  0.38860407
neck_girth        418  0.016311646  1.5304298   0.5129456  0.

In [22]:
column_curves_measures = {}

for idx, gender in enumerate(genders):

    template = subdivided_bodies['faces'][gender].to(device)

    column_curves_measures[gender] = []
    best_neck = gender_curves[gender][curve_index['neck_girth']][gender_results[gender]['best']['neck_girth']]
    best_neck = tc.FloatTensor(best_neck).to(device)

    best_waist = gender_curves[gender][curve_index['waist_girth']][gender_results[gender]['best']['waist_girth']]
    best_waist = tc.FloatTensor(best_waist).to(device)
    print(best_neck.shape, best_waist.shape)
    
    bodies_interator = tqdm(subdivided_bodies['vertices'][gender])
    for body in bodies_interator:

        best_neck_pos = CurveUtils.generate_positions(best_neck, body.to(device))
        best_waist_pos = CurveUtils.generate_positions(best_waist, body.to(device))

        neck_ht = best_neck_pos[:,1].max()
        waist_ht = best_waist_pos[:,1].max()

        neck_width = abs(best_neck_pos[:,2]).max()
        waist_width = abs(best_waist_pos[:,2]).max()

        
        width = (neck_width + (waist_width/2))*1.2
        height = neck_ht - waist_ht
        center = (0, waist_ht + (height/2), -width/2)
        column_box = BBox(length=width, width=width, height=height, center=tc.FloatTensor(center).to(device))
        column_curves = CurveGenerator.calculate_curve(body.to(device), column_box, 0, template, device)      


        all_col_curves = []
        for curve in column_curves[0]:
            positions = CurveUtils.generate_positions(tc.FloatTensor(curve).to(device), body.to(device))
            all_col_curves.append(positions[:,0].abs().mean())
        idmin = tc.FloatTensor(all_col_curves).abs().argmin()
        if bodies_interator.n == 29 and gender == "male":
            save_obj("neckwaist.obj", tc.FloatTensor(column_curves[2][idmin]))
            column_box.save_limits('bb.obj')
        measure = CurveUtils.linear_distance(tc.FloatTensor(column_curves[2][idmin]))
        column_curves_measures[gender].append(measure)

torch.Size([455, 6]) torch.Size([610, 6])


  3%|▎         | 2/72 [00:02<01:27,  1.25s/it]


KeyboardInterrupt: 

In [12]:
errors = abs(column_curves_measures['male'] - (mfd_gender_measures['male']['neck_waist']/10))
gender_results['male'].loc['neck_waist'] = 0, errors.min(), errors.max(), errors.mean(), errors.std()

errors = abs(column_curves_measures['female'] - (mfd_gender_measures['female']['neck_waist']/10))
gender_results['female'].loc['neck_waist'] = 0, errors.min(), errors.max(), errors.mean(), errors.std()

In [13]:
(gender_results['male']*10).astype(float).round(decimals=2)

,best,min,max,mean,std
bust_chest_girth,190.0,0.48,49.17,13.69,10.69
waist_girth,330.0,0.08,21.72,6.31,5.44
hip_girth,130.0,0.02,12.59,4.56,3.61
thigh_girth,540.0,0.04,18.03,7.03,5.11
upper_arm_girth,380.0,0.03,7.72,3.18,2.10
neck_girth,4960.0,0.03,24.17,7.39,5.78
neck_waist,0.0,0.03,33.70,8.65,7.04


In [14]:
(gender_results['female']*10).astype(float).round(decimals=2)

,best,min,max,mean,std
bust_chest_girth,140.0,1.61,47.39,18.62,11.97
waist_girth,300.0,0.15,36.63,10.97,9.47
hip_girth,100.0,0.57,54.28,12.66,11.80
thigh_girth,470.0,0.20,32.77,8.69,6.83
upper_arm_girth,340.0,0.31,16.41,6.15,3.89
neck_girth,4180.0,0.16,15.30,5.13,3.63
neck_waist,0.0,0.26,37.33,12.09,8.24


In [15]:
best_gender_curves = dict()
for gender in ['male']:
    all_positions = []
    best_gender_curves[gender] = []
    for index, curve in enumerate(our_semantic[:-2]):
        best = gender_results[gender].loc[curve]['best']
        coordinates = gender_curves[gender][curve_index[curve]][best]
        best_gender_curves[gender].append(coordinates)
        faces = subdivided_bodies['faces'][gender].to(device)
        bodies = subdivided_bodies['vertices'][gender]
        body = bodies[29].to(device)
        position = CurveUtils.generate_positions(tc.FloatTensor(coordinates).to(device), body.to(device))
        all_positions.append(position)
    save_obj(f'output/{gender}_points.obj', tc.row_stack(all_positions))
    save_obj(f'output/{gender}_body.obj', body, template+1)
tc.save(best_gender_curves, 'data/selected_gender_curves.zip')

In [16]:
gender_results[gender]

,best,min,max,mean,std
bust_chest_girth,19,0.048316956,4.917015,1.3688946,1.0689367
waist_girth,33,0.008003235,2.1719513,0.6310151,0.5437107
hip_girth,13,0.0023117065,1.2593384,0.45648277,0.3605611
thigh_girth,54,0.004257202,1.8026123,0.70295495,0.5112756
upper_arm_girth,38,0.002937317,0.7723694,0.318335,0.2103261
neck_girth,496,0.0025863647,2.4173927,0.7392074,0.5782629
neck_waist,0.0,0.003262,3.370305,0.865489,0.704368


In [17]:
best_gender_measures = dict()
for idx, gender in enumerate(genders):
    best_gender_measures[gender] = []
    for index, curve in enumerate(our_semantic[:-2]):
        best = gender_results[gender].loc[curve]['best']
        measured = gender_measures[gender][index][best].numpy()
        best_gender_measures[gender].append(measured*10)
    best_gender_measures[gender].append(tc.arange(2).repeat(36).numpy()+1)
    best_gender_measures[gender].append([gender]*72)
    best_gender_measures[gender].append(["our"]*72)
    best_gender_measures[gender].append(mfd_gender_measures[gender].index)

In [18]:
for idx, gender in enumerate(genders):
    best_gender_measures[gender].insert(6, np.stack(column_curves_measures[gender])*10)
    # print(tc.row_stack(column_curves_measures[gender]).shape)
    # print(best_gender_measures[gender])

In [19]:
aditional_semantic = ['repetition', "gender", "measures_station", 'subject']
our_measures = pd.concat([
    pd.DataFrame(best_gender_measures['female'], index=our_semantic[:-1]+aditional_semantic).T,
    pd.DataFrame(best_gender_measures['male'], index=our_semantic[:-1]+aditional_semantic).T
])

our_measures.to_pickle("data/our_measures.zip")

In [20]:
our_measures

,bust_chest_girth,waist_girth,hip_girth,thigh_girth,upper_arm_girth,neck_girth,neck_waist,repetition,gender,measures_station,subject
0,879.825195,750.751343,1053.165405,626.480164,297.009918,329.085876,411.729614,1,female,our,IEEEP2_01
1,883.476624,752.710327,1055.366943,623.614441,298.111176,325.854523,419.242859,2,female,our,IEEEP2_01
2,896.443237,682.851685,948.50531,535.620667,276.295074,316.661316,412.113953,1,female,our,IEEEP2_03
3,882.15509,665.351685,952.023193,535.229431,276.825439,312.521088,406.582611,2,female,our,IEEEP2_03
4,1098.579834,980.695679,1137.411987,669.249634,347.686432,372.356293,431.70166,1,female,our,IEEEP2_07
...,...,...,...,...,...,...,...,...,...,...,...
67,992.585754,758.534485,950.535889,576.500366,321.028198,366.693695,425.313965,2,male,our,IEEEP2_67
68,1241.935059,1257.259766,1150.945557,669.063599,362.176331,444.99823,492.719849,1,male,our,IEEEP2_70
69,1232.241089,1269.487793,1153.158325,663.742371,361.474792,443.42807,492.745331,2,male,our,IEEEP2_70
70,961.500183,886.61554,991.524658,554.797913,317.637756,387.154846,439.564789,1,male,our,IEEEP2_72
